In [1]:
%%time
import boto3
# SUBO MI CODIGO PYTHON EN UNA RUTA ESPECIFICA

BUCKET_NAME = 'notification.output.test'
AWS_DEFAULT_ACL = 'public-read'
REGION = 'us-west-1'
path_in_pc = '/home/rmg/STORAGE/tests_courses_research/practice/GLUE/campaing/code_up_campaing.py'
path_s3 = 'proccess.py'

client = boto3.client('s3')
s3 = boto3.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

if not bucket.creation_date:
    s3.create_bucket(
        ACL=AWS_DEFAULT_ACL,
        Bucket=BUCKET_NAME,
        CreateBucketConfiguration={'LocationConstraint': REGION}
    )
    
client.upload_file(
    path_in_pc,
    BUCKET_NAME,
    path_s3,
    ExtraArgs={
        'ContentType': 'text/x-python',
        'ACL': AWS_DEFAULT_ACL
    }
)

CPU times: user 269 ms, sys: 16 ms, total: 285 ms
Wall time: 1.44 s


In [10]:
%%time
import boto3
# PARA EJECUTAR UNA SOLA VEZ
# Creando el job GLUE
job_name = 'copy-json-glue'
BUCKET_NAME = 'notification.job1'
AWS_DEFAULT_ACL = 'public-read'
REGION = 'us-west-1'
path_in_pc = '/home/rmg/STORAGE/tests_courses_research/practice/GLUE/campaing/code_up_campaing.py'
path_s3 = 'proccess.py'

client_glue = boto3.client('glue')
response_glue = client_glue.create_job(
    Name=job_name,
    Description='Prueba de copias csv en flujo GLUE',
    Role='Glue_role',
    ExecutionProperty={
        'MaxConcurrentRuns': 3
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': 's3://{}/{}'.format(BUCKET_NAME, path_s3)
    }
)
response_glue

CPU times: user 54.3 ms, sys: 3.73 ms, total: 58 ms
Wall time: 708 ms


In [11]:
# RUN JOB GLUE
response_run = client_glue.start_job_run(
    JobName=job_name
)
response_run

{'JobRunId': 'jr_3f165120b1e25d08ed50841ad740a534ae3059e99766930623ec549684f42b72',
 'ResponseMetadata': {'RequestId': '1d51e782-57fd-11e9-9204-47b18bf43c9b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Apr 2019 23:47:04 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1d51e782-57fd-11e9-9204-47b18bf43c9b'},
  'RetryAttempts': 0}}

In [12]:
job_run_id = response_run.get('JobRunId', '')
job_run_id

'jr_3f165120b1e25d08ed50841ad740a534ae3059e99766930623ec549684f42b72'

In [13]:
import time
status = 'RUNNING'

while status == 'RUNNING':
    response_status = client_glue.get_job_run(
        JobName=job_name,
        RunId = job_run_id
    )
    status = response_status.get('JobRun', {}).get('JobRunState', {})
    print(status, end=' ')
    if status != 'RUNNING':
        print("\n ", response_status)
        break
        
    time.sleep(10)

RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING SUCCEEDED 
  {'JobRun': {'Id': 'jr_3f165120b1e25d08ed50841ad740a534ae3059e99766930623ec549684f42b72', 'Attempt': 0, 'JobName': 'copy-json-glue', 'StartedOn': datetime.datetime(2019, 4, 5, 18, 47, 4, 232000, tzinfo=tzlocal()), 'LastModifiedOn': datetime.datetime(2019, 4, 5, 18, 51, 56, 436000, tzinfo=tzlocal()), 'CompletedOn': datetime.datetime(2019, 4, 5, 18, 51, 56, 436000, tzinfo=tzlocal()), 'JobRunState': 'SUCCEEDED', 'PredecessorRuns': [], 'AllocatedCapacity': 10, 'ExecutionTime': 118, 'Timeout': 2880, 'LogGroupName': '/aws-glue/jobs'}, 'ResponseMetadata': {'RequestId': 'cef73291-57fd-11e9-964b-05ff9084c187', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 05 Apr 2019 23:52:02 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '37

In [8]:
!aws s3 ls s3://notification.job1

2019-04-05 18:35:54       1637 proccess.py
